In [3]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Библиотеки sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

# Импортируем РАЗНЫЕ модели
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

# ==========================================
# 1. ЗАГРУЗКА И ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ
# ==========================================
print("Загрузка данных...")
df = pd.read_csv('./src/weatherAUS.csv')

# Удаляем строки, где неизвестен ответ (RainToday)
df.dropna(subset=['RainToday'], inplace=True)

# ==========================================
# 2. ИНЖИНИРИНГ ПРИЗНАКОВ (Сценарий "9 утра")
# ==========================================

# Список колонок, которые удаляются, потому что это "будущее" или "ответ":
cols_to_drop = [
    'Date', 'Location',        # Локация может быть полезна, но для упрощения уберем
    'RainTomorrow',            # Это прогноз на завтра, нам не нужен
    'Rainfall',                # Это прямая подсказка (ответ), удаляем обязательно!
    'Evaporation', 'Sunshine', # Данные за весь день
    'MaxTemp',                 # Максимум дня (еще не наступил)
    'RISK_MM',                 # В некоторых версиях датасета есть этот столбец-спойлер
    
    # Удаляем все вечерние замеры (3pm), так как утром их у нас нет
    'WindGustSpeed', 'WindGustDir', # Порывы ветра обычно за день
    'WindDir3pm', 'WindSpeed3pm', 'Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'Temp3pm'
]
actual_cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df_morning = df.drop(columns=actual_cols_to_drop)

print(f"Признаки для обучения ({len(df_morning.columns)}):")
print(df_morning.columns.tolist())

# ==========================================
# 3. ПРЕДОБРАБОТКА (Encoding & Scaling)
# ==========================================

# Кодируем целевую переменную (RainToday: Yes->1, No->0)
le = LabelEncoder()
df_morning['RainToday'] = le.fit_transform(df_morning['RainToday'])

X = df_morning.drop('RainToday', axis=1)
y = df_morning['RainToday']

# Разделяем признаки по типу
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Заполняем пропуски
imputer_num = SimpleImputer(strategy='median')
X[numerical_cols] = imputer_num.fit_transform(X[numerical_cols])

imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = imputer_cat.fit_transform(X[categorical_cols])

# One-Hot Encoding
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Масштабирование (Нормализация 0-1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Разделение на Train / Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"\nРазмер обучающей выборки: {X_train.shape}")

# ==========================================
# 4. ОБУЧЕНИЕ НЕСКОЛЬКИХ МОДЕЛЕЙ
# ==========================================

# Словарь с моделями, которые нужно проверить
models = {
    "Logistic Regression": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced', n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
}

best_model = None
best_auc = 0
best_name = ""

print("\n--- НАЧИНАЕМ СОРЕВНОВАНИЕ МОДЕЛЕЙ ---")

for name, model in models.items():
    print(f"\nОбучение {name}...")
    model.fit(X_train, y_train)
    
    # Предсказание
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] # Вероятность класса 1
    
    # Метрики  
    
    # Выбрал ROC-AUC (Area Under Curve) и Accuracy:
    # Почему не просто Accuracy? В Австралии много солнечных дней. Если модель будет всегда говорить "Дождя не будет", она может получить Accuracy 80%, но она будет бесполезна.
    # ROC-AUC показывает, насколько хорошо модель умеет ранжировать примеры (отличать класс 0 от класса 1) независимо от порога принятия решения.
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    print(f"--> Accuracy: {acc:.4f}")
    print(f"--> ROC-AUC:  {auc:.4f}")
    
    # Сравниваем с лучшей на данный момент
    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_name = name

# ==========================================
# 5. ИТОГИ И СЕРИАЛИЗАЦИЯ
# ==========================================
print("\n" + "="*30)
print(f"ПОБЕДИТЕЛЬ: {best_name}")
print(f"С метрикой ROC-AUC: {best_auc:.4f}")
print("="*30)

# Выводим подробный отчет для победителя
print(f"\nОтчет классификации для {best_name}:")
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))


# Важность признаков (Что больше всего влияет на решение брать зонт?)
feature_importances = pd.DataFrame(best_model.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print("\nТоп-5 признаков, почему идет дождь:")
print(feature_importances.head(5))

# Задаем имя папки
save_dir = './models'

# Создаем папку, если её нет
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"Папка '{save_dir}' создана.")

# Сохраняем файлы ВНУТРЬ этой папки
joblib.dump(best_model, os.path.join(save_dir, 'umbrella_model.pkl'))
joblib.dump(scaler, os.path.join(save_dir, 'umbrella_scaler.pkl'))
joblib.dump(X.columns.tolist(), os.path.join(save_dir, 'model_columns.pkl'))

print(f"\nВсе файлы успешно сохранены в папку: {os.path.abspath(save_dir)}")

Загрузка данных...
Признаки для обучения (8):
['MinTemp', 'WindDir9am', 'WindSpeed9am', 'Humidity9am', 'Pressure9am', 'Cloud9am', 'Temp9am', 'RainToday']

Размер обучающей выборки: (113759, 21)

--- НАЧИНАЕМ СОРЕВНОВАНИЕ МОДЕЛЕЙ ---

Обучение Logistic Regression...
--> Accuracy: 0.7466
--> ROC-AUC:  0.8333

Обучение Random Forest...
--> Accuracy: 0.7700
--> ROC-AUC:  0.8466

Обучение Gradient Boosting...
--> Accuracy: 0.8331
--> ROC-AUC:  0.8560

ПОБЕДИТЕЛЬ: Gradient Boosting
С метрикой ROC-AUC: 0.8560

Отчет классификации для Gradient Boosting:
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     22016
           1       0.69      0.47      0.56      6424

    accuracy                           0.83     28440
   macro avg       0.78      0.70      0.73     28440
weighted avg       0.82      0.83      0.82     28440


Топ-5 признаков, почему идет дождь:
              importance
Humidity9am     0.463965
MinTemp         0.148218
Pressure9